In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix, SCORERS, classification_report
import numpy as np

In [ ]:
featuresCD = pd.read_csv("/content/drive/MyDrive/NYC DSA Capstone Project/Data/featuresCD.csv", low_memory = False)
featuresCD.drop(["Unnamed: 0"], axis = 1, inplace = True)
featuresCD

,loan_status_trim,state_income,annual_inc_joint,indiv_app,dti_joint,days_from_1st_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,home_ownership,inq_fi,inq_last_12m,installment,int_rate,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,for_debt_consol,total_acc,revol_bal,revol_util,delinq_amnt,delinq_2yrs,mths_since_last_delinq,mths_since_last_major_derog,grade
0,0,2,55000.0,1,5.91,4505.0,10,679.0,675.0,3600.0,1,3.0,4.0,123.03,13.99,3600.0,7.0,0.0,0.0,1,13.0,2765.0,29.7,0.0,0.0,30.0,30.0,4
1,0,1,65000.0,1,16.06,5844.0,10,719.0,715.0,24700.0,1,0.0,6.0,820.28,11.99,24700.0,22.0,0.0,0.0,0,38.0,21470.0,19.2,0.0,1.0,6.0,0.0,4
2,0,2,34000.0,1,10.20,10288.0,4,694.0,690.0,11950.0,0,0.0,0.0,405.18,13.44,11950.0,5.0,0.0,0.0,1,6.0,8822.0,68.4,0.0,0.0,0.0,0.0,4
3,0,2,64000.0,1,34.95,7122.0,3,704.0,700.0,1400.0,1,0.0,0.0,47.10,12.88,1400.0,17.0,0.0,0.0,0,24.0,37828.0,67.2,0.0,0.0,0.0,0.0,4
4,0,2,112000.0,1,8.68,8065.0,10,804.0,800.0,18000.0,1,0.0,1.0,400.31,11.99,18000.0,17.0,0.0,0.0,1,27.0,10711.0,15.5,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583975,0,2,119000.0,1,32.73,8827.0,10,689.0,685.0,23800.0,1,4.0,10.0,559.85,14.49,23800.0,13.0,0.0,0.0,1,29.0,107747.0,89.5,0.0,0.0,0.0,0.0,4
583976,0,3,180792.0,1,22.03,5328.0,9,709.0,705.0,29400.0,1,3.0,2.0,683.94,13.99,29400.0,16.0,0.0,0.0,1,32.0,77480.0,85.2,0.0,0.0,0.0,0.0,4
583977,1,2,157000.0,1,10.34,1949.0,3,739.0,735.0,32000.0,1,2.0,0.0,752.74,14.49,32000.0,14.0,0.0,0.0,0,18.0,111598.0,27.4,0.0,0.0,0.0,0.0,4
583978,0,2,150000.0,1,12.25,7001.0,10,669.0,665.0,16000.0,0,1.0,1.0,362.34,12.79,16000.0,12.0,4.0,3.0,0,28.0,7700.0,55.0,0.0,0.0,68.0,68.0,4


In [ ]:
featuresCD.columns

In [ ]:
data = featuresCD.iloc[:, 1:]
target = featuresCD.iloc[:, 0]

In [ ]:
data

,state_income,annual_inc_joint,indiv_app,dti_joint,days_from_1st_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,home_ownership,inq_fi,inq_last_12m,installment,int_rate,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,for_debt_consol,total_acc,revol_bal,revol_util,delinq_amnt,delinq_2yrs,mths_since_last_delinq,mths_since_last_major_derog,grade
0,2,55000.0,1,5.91,4505.0,10,679.0,675.0,3600.0,1,3.0,4.0,123.03,13.99,3600.0,7.0,0.0,0.0,1,13.0,2765.0,29.7,0.0,0.0,30.0,30.0,4
1,1,65000.0,1,16.06,5844.0,10,719.0,715.0,24700.0,1,0.0,6.0,820.28,11.99,24700.0,22.0,0.0,0.0,0,38.0,21470.0,19.2,0.0,1.0,6.0,0.0,4
2,2,34000.0,1,10.20,10288.0,4,694.0,690.0,11950.0,0,0.0,0.0,405.18,13.44,11950.0,5.0,0.0,0.0,1,6.0,8822.0,68.4,0.0,0.0,0.0,0.0,4
3,2,64000.0,1,34.95,7122.0,3,704.0,700.0,1400.0,1,0.0,0.0,47.10,12.88,1400.0,17.0,0.0,0.0,0,24.0,37828.0,67.2,0.0,0.0,0.0,0.0,4
4,2,112000.0,1,8.68,8065.0,10,804.0,800.0,18000.0,1,0.0,1.0,400.31,11.99,18000.0,17.0,0.0,0.0,1,27.0,10711.0,15.5,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583975,2,119000.0,1,32.73,8827.0,10,689.0,685.0,23800.0,1,4.0,10.0,559.85,14.49,23800.0,13.0,0.0,0.0,1,29.0,107747.0,89.5,0.0,0.0,0.0,0.0,4
583976,3,180792.0,1,22.03,5328.0,9,709.0,705.0,29400.0,1,3.0,2.0,683.94,13.99,29400.0,16.0,0.0,0.0,1,32.0,77480.0,85.2,0.0,0.0,0.0,0.0,4
583977,2,157000.0,1,10.34,1949.0,3,739.0,735.0,32000.0,1,2.0,0.0,752.74,14.49,32000.0,14.0,0.0,0.0,0,18.0,111598.0,27.4,0.0,0.0,0.0,0.0,4
583978,2,150000.0,1,12.25,7001.0,10,669.0,665.0,16000.0,0,1.0,1.0,362.34,12.79,16000.0,12.0,4.0,3.0,0,28.0,7700.0,55.0,0.0,0.0,68.0,68.0,4


In [ ]:
target

0         0
1         0
2         0
3         0
4         0
         ..
583975    0
583976    0
583977    1
583978    0
583979    1
Name: loan_status_trim, Length: 583980, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, stratify = target, shuffle = True)

In [ ]:
%%time

print("-"*60)
tree_model = tree.DecisionTreeClassifier(criterion = "entropy",
                                         class_weight = {0:2e-8, 1:1},
                                         #min_sample 
                                         random_state = 0)

tree_model.fit(X_train, y_train)

print(f"Training score: {tree_model.score(X_train, y_train)}")
print(f"Test score:     {tree_model.score(X_test, y_test)}")

print("-"*60)

print(f"Training Confusion Matrix")
print(confusion_matrix(y_true = y_train,
                       y_pred = tree_model.predict(X_train)))

print("-"*60)

print(f"Testing Confusion Matrix")
print(confusion_matrix(y_true = y_test,
                       y_pred = tree_model.predict(X_test)))

print("-"*60)

In [ ]:
pd.DataFrame(list(zip(featuresCD.columns, tree_model.feature_importances_)), columns = ["feature", "importance"]).sort_values(by = "importance", ascending = False)

Look at Gradient Boosting again

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, stratify = target, shuffle = True)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.model_selection as ms

In [ ]:
param_grid = {
    "learning_rate" : [0.01],
    "max_depth"     : [15],
    "subsample"     : [0.9],
    "n_estimators"  : [100,]
}

In [ ]:
%%time
gbm = GradientBoostingClassifier()
grid_search_gbm = ms.GridSearchCV(gbm, param_grid, scoring = 'recall', cv = 3, n_jobs = -1)
grid_search_gbm.fit(X_train, y_train)

CPU times: user 20min 14s, sys: 8.03 s, total: 20min 23s
Wall time: 37min 36s


In [ ]:
grid_search_gbm.best_params_

{'learning_rate': 0.01, 'max_depth': 15, 'n_estimators': 100, 'subsample': 0.9}

In [ ]:
grid_search_gbm.best_score_

0.003748242659491499

In [ ]:
%%time
print("-"*60)
print("Training score:",grid_search_gbm.best_estimator_.score(X_train, y_train))
print("Testing score:", grid_search_gbm.best_estimator_.score(X_test, y_test))

print("-"*60)

print(f"Training Confusion Matrix")
print(confusion_matrix(y_true = y_train,
                       y_pred = grid_search_gbm.best_estimator_.predict(X_train)))

print("-"*60)

print(f"Testing Confusion Matrix")
print(confusion_matrix(y_true = y_test,
                       y_pred = grid_search_gbm.best_estimator_.predict(X_test)))

print("-"*60)

------------------------------------------------------------
Training score: 0.7692814822425426
Testing score: 0.7484074797082092
------------------------------------------------------------
Training Confusion Matrix
[[349529      0]
 [107788   9867]]
------------------------------------------------------------
Testing Confusion Matrix
[[87288    94]
 [29291   123]]
------------------------------------------------------------
CPU times: user 22.2 s, sys: 60.2 ms, total: 22.2 s
Wall time: 22.1 s


In [ ]:
grid_search_gbm.cv_results_

In [ ]:
# Can't use class_weight for Gradient Boosting

class_weight = {0:1, 
                1: len(featuresCD[featuresCD["loan_status_trim"] == 0]) / len(featuresCD[featuresCD["loan_status_trim"] == 1])
                }

In [ ]:
from sklearn.svm import SVC

In [ ]:
%%time 

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size = 0.01, test_size = 0.01, stratify = target, shuffle = True)

svm = SVC()

weights = np.linspace(2e-16, 3, 11)

param_grid_svm = {
    "kernel": ["sigmoid"],
    "class_weight" : [{0:x, 1:3-x} for x in weights]
    }

grid_search_svm = ms.GridSearchCV(svm, 
                                  param_grid_svm, 
                                  scoring = "f1_macro", 
                                  cv = 2, 
                                  n_jobs = -1)

grid_search_svm.fit(X_train, y_train)
print("-"*100)
print("Training score:",grid_search_svm.best_estimator_.score(X_train, y_train))
print("Testing score:",grid_search_svm.best_estimator_.score(X_test, y_test))
print("-"*100)
print("Training")
print(confusion_matrix(y_true = y_train, y_pred = grid_search_svm.best_estimator_.predict(X_train)))
print("-"*100)
print("Validation")
print(confusion_matrix(y_true = y_test, y_pred = grid_search_svm.best_estimator_.predict(X_test)))
# print(classification_report(y_test, y_pred = grid_search_svm.best_estimator_.predict(X_test)))
# print(grid_search_svm.best_params_)

----------------------------------------------------------------------------------------------------
Training score: 0.6377804418564823
Testing score: 0.6421232876712328
----------------------------------------------------------------------------------------------------
Training
[[3387  982]
 [1133  337]]
----------------------------------------------------------------------------------------------------
Validation
[[3438  931]
 [1159  312]]
CPU times: user 5.76 s, sys: 86.1 ms, total: 5.84 s
Wall time: 11.8 s


In [ ]:
%%time 

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size = 0.01, test_size = 0.01, stratify = target, shuffle = True)

svm = SVC(class_weight = {0:1.2, 1:1.8}, kernel = "sigmoid", C = 33)

param_grid_svm = {
    "C":[33]
    }

grid_search_svm = ms.GridSearchCV(svm, 
                                  param_grid_svm, 
                                  scoring = "f1_macro", 
                                  cv = 2, 
                                  n_jobs = -1)

grid_search_svm.fit(X_train, y_train)
print("-"*100)
print(grid_search_svm.best_estimator_.score(X_train, y_train))
print(grid_search_svm.best_estimator_.score(X_test, y_test))
print("-"*100)
print("Training")
print(confusion_matrix(y_true = y_train, y_pred = grid_search_svm.best_estimator_.predict(X_train)))
print("Validation")
print(confusion_matrix(y_true = y_test, y_pred = grid_search_svm.best_estimator_.predict(X_test)))
#print(classification_report(y_test, y_pred = grid_search_svm.best_estimator_.predict(X_test)))
#print(grid_search_svm.best_params_)
#print(grid_search_svm.cv_results_)

----------------------------------------------------------------------------------------------------
0.5771536221955814
0.5986301369863014
----------------------------------------------------------------------------------------------------
Training
[[2890 1479]
 [ 990  480]]
Validation
[[2988 1381]
 [ 963  508]]
CPU times: user 5.53 s, sys: 74.9 ms, total: 5.6 s
Wall time: 6.97 s


In [ ]:
import imblearn
from imblearn.over_sampling import SVMSMOTE
from imblearn.pipeline import make_pipeline

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
%%time 

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size = 0.01, test_size = 0.01, stratify = target, shuffle = True)

svm = SVC(#class_weight = {0:1.2, 1:1.8},
          kernel = "sigmoid", 
          C = 33)
          #tol = 2e-16, 
          #shrinking = False
          #break_ties = True)

pipe = make_pipeline(
    SVMSMOTE(svm_estimator = svm,
             sampling_strategy = {0:4369, 1:4369},
             n_jobs = -1,
             m_neighbors = 1,
             k_neighbors = 2),
    svm
)

param_grid_svm = {
    
    }

pipe_grid_search_svm = ms.GridSearchCV(pipe, 
                                  param_grid_svm, 
                                  scoring = "f1_macro", 
                                  cv = 2, 
                                  n_jobs = -1)

pipe_grid_search_svm.fit(X_train, y_train)
print(pipe_grid_search_svm.best_estimator_.score(X_train, y_train))
print(pipe_grid_search_svm.best_estimator_.score(X_test, y_test))
print(confusion_matrix(y_true = y_train, y_pred = pipe_grid_search_svm.best_estimator_.predict(X_train)))
print(confusion_matrix(y_true = y_test, y_pred = pipe_grid_search_svm.best_estimator_.predict(X_test)))
print(classification_report(y_test, y_pred = pipe_grid_search_svm.best_estimator_.predict(X_test)))
#print(pipe_grid_search_svm.best_params_)
print(pipe_grid_search_svm.cv_results_)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

0.4723411543072444
0.4808219178082192
[[1945 2424]
 [ 657  813]]
[[1971 2398]
 [ 634  837]]
              precision    recall  f1-score   support

           0       0.76      0.45      0.57      4369
           1       0.26      0.57      0.36      1471

    accuracy                           0.48      5840
   macro avg       0.51      0.51      0.46      5840
weighted avg       0.63      0.48      0.51      5840

{'mean_fit_time': array([5.9787401]), 'std_fit_time': array([0.11725581]), 'mean_score_time': array([1.18908536]), 'std_score_time': array([0.02192986]), 'params': [{}], 'split0_test_score': array([0.49076641]), 'split1_test_score': array([0.48693074]), 'mean_test_score': array([0.48884858]), 'std_test_score': array([0.00191783]), 'rank_test_score': array([1], dtype=int32)}
CPU times: user 15.3 s, sys: 63 ms, total: 15.3 s
Wall time: 23.4 s


In [ ]:
print("Imbalanced-Learn", imblearn.__version__)

In [ ]:
sorted(SCORERS.keys())

In [ ]:
grid_search_svm.get_params().keys()

In [ ]:
dir(grid_search_svm)

In [ ]:
grid_search_svm.cv_results_